### Data Loading

Este notebook va a ser utilizado para carga el data set ya limpio y con todos los datos necesarios, originalmente scrapeado. Será cargado a MySQL Workbench.

In [1]:
import mysql.connector as conn
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
with open('contraseñas.txt') as file:
    mypasswd = file.read()

In [3]:
# Nos conectamos al servidor 
conexion = conn.connect(host='localhost',
                        user='root',
                        passwd=f'{mypasswd}'
                        )

cursor  = conexion.cursor()

cursor

In [4]:
cursor.execute('drop database if exists orcas')

cursor.execute('create database orcas;')

In [5]:
conexion = conn.connect(host='localhost',
                        user='root',
                        passwd=f'{mypasswd}',
                        database='orcas'
                        )

cursor  = conexion.cursor()

In [6]:
c = cursor.execute
c

<bound method CMySQLCursor.execute of <mysql.connector.cursor_cext.CMySQLCursor object at 0x000001E8FFC29900>>

#### Carga del dataset
Leemos y cargamos el archivo CSV que hemos limpiado y completado en pasos anteriores.

In [12]:
df_full = pd.read_csv('../data/full_data.csv')
df_full.head()

,Unnamed: 0,date,Latitude,Longitude,Followed_GTOA_Protocol,Interaction_time,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,...,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour
0,0,2023-11-01 22:15:00,32.791633,-9.906633,No,0-10,Sail,10-12.5,No,No,...,Over 10,200m+,On,On,White/light,Blue,No,No,"Orca interaction at 10:15pm on 01/11, 40 miles...",I would describe the behaviour of the Orca dur...
1,1,2023-10-31 07:50:00,39.433333,-9.383333,Yes,0-10,Sail,12.5-15,No,No,...,2 - 5,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",No,We had sandbags on our sugar scoops and metal ...,Juveniles hitting the rudders adults close by
2,2,2023-09-19 11:00:00,37.666667,-8.900000,No,0-10,Sail,12.5-15,No,No,...,2 - 5,40 - 200m,On,On,White/light,Other,"Yes, moderate - immediate repairs required",No,We saw the orca approach from 10 o’clock posit...,There was an initial approach 45 minutes earli...
3,3,2023-09-01 13:15:00,45.600000,-3.750000,Yes,10-30,Sail,15+,No,Yes,...,Over 10,200m+,Off,Off,White/light,Black,"Yes, moderate - immediate repairs required",No,Les trois orques passent constamment de bâbord...,Pas de comportement visblement agressif./// No...
4,4,2023-09-02 03:45:00,42.750000,-9.233333,Yes,0-10,Sail,12.5-15,No,No,...,5 - 10,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",Yes,Arrêt du pilote automatique a la 2 eme interac...,Approche furtive à la première interaction dir...


In [13]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     153 non-null    int64  
 1   date                           153 non-null    object 
 2   Latitude                       153 non-null    float64
 3   Longitude                      153 non-null    float64
 4   Followed_GTOA_Protocol         153 non-null    object 
 5   Interaction_time               153 non-null    object 
 6   Boat_Type                      153 non-null    object 
 7   Boat_Length                    153 non-null    object 
 8   Towing_Inflatable              153 non-null    object 
 9   Trailing_Fishing_Lure          153 non-null    object 
 10  Physical_Contact_With_Boat     153 non-null    object 
 11  Number_of_Adult_Orcas          153 non-null    int64  
 12  Number_of_Juvenile_Orcas       153 non-null    int

In [17]:
# En el anterior notebook añadimos a propósito el índice de cada fila con intención de hacerlo el Primary Key de cada interacción. Vamos a ponerle de titulo 'Indice':
df_full.rename(columns={'Indice': 'indice'}, inplace=True)
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   indice                         153 non-null    int64         
 1   date                           153 non-null    datetime64[ms]
 2   Latitude                       153 non-null    float64       
 3   Longitude                      153 non-null    float64       
 4   Followed_GTOA_Protocol         153 non-null    object        
 5   Interaction_time               153 non-null    object        
 6   Boat_Type                      153 non-null    object        
 7   Boat_Length                    153 non-null    object        
 8   Towing_Inflatable              153 non-null    object        
 9   Trailing_Fishing_Lure          153 non-null    object        
 10  Physical_Contact_With_Boat     153 non-null    object        
 11  Number_of_Adult_Orc

In [18]:
# Volvemos a cambiar la fecha a datetime, parece que al guardar a csv y cargar el dataframe de nuevo, se desconfigura este datatype:
df_full.date = df_full.date.astype('datetime64[ms]')
print(df_full.date.dtype)

datetime64[ms]


#### Creación de la tabla dentro de la BBDD

In [20]:
# Como debo definir el tipo de dato de cada columna, me gustaría calcular la longitud máxima de caracteres de cada una de las columnas que tengan datos de tipo objeto.

# Lo hago a continuación:

max_lengths = df_full.select_dtypes(include='object').apply(lambda col: col.astype(str).apply(len).max())
max_lengths

Followed_GTOA_Protocol           7
Interaction_time                 7
Boat_Type                       14
Boat_Length                      7
Towing_Inflatable                7
Trailing_Fishing_Lure            7
Physical_Contact_With_Boat       7
Rudder                          11
Motoring_or_Sailing             12
Speed_Knots                      7
Sea_State                        8
Wind_Speed_Beaufort             21
Daylight_or_Darkness             5
Cloud_Cover                      9
Distance_Off_Land_NM             7
Depth_Meters                     9
Depth_Gauge                      7
Autopilot                        7
Hull_Topsides_Color             11
Antifoul_Color                  10
Boat_Damaged                    50
Tow_Required                     7
Crew_Response                 6163
Orcas_Behaviour               1633
dtype: int64

* Como se puede ver, debido a la limpieza llevada a cabo en pasos anteriores la mayoría de columnas tienen un número de caracteres reducida, a salvedad de los comentarios de la tripulación.

In [22]:
c('drop table if exists interactions;')

query ='''

create table interactions(
    indice int primary key not null,
    date datetime,
    Latitude float,
    Longitude float,
    Followed_GTOA_Protocol varchar(10),
    Interaction_time varchar(10),
    Boat_Type varchar(15),
    Boat_Length varchar(10),
    Towing_Inflatable varchar(10),
    Trailing_Fishing_Lure varchar(10),
    Physical_Contact_With_Boat varchar(10),
    Number_of_Adult_Orcas int,
    Number_of_Juvenile_Orcas int,
    Number_of_Uncertain_Age_Orcas int,
    Rudder varchar(15),
    Motoring_or_Sailing varchar(15),
    Speed_Knots varchar(10),
    Sea_State varchar(10),
    Wind_Speed_Beaufort varchar(25),
    Daylight_or_Darkness varchar(10),
    Cloud_Cover varchar(10),
    Distance_Off_Land_NM varchar(10),
    Depth_Meters varchar(15),
    Depth_Gauge varchar(10),
    Autopilot varchar(10),
    Hull_Topsides_Color varchar(15),
    Antifoul_Color varchar(15),
    Boat_Damaged varchar(60),
    Tow_Required varchar(10),
    Crew_Response varchar(6200),
    Orcas_Behaviour varchar(1700)
    );
'''

c(query)


#### Carga de la Tabla con nuestros datos
Tenemos la tabla con sus columnas y respectivos tipos de dato cargados. Pasamos a rellenar la tabla con nuestra dataframe.

In [23]:
nombre_tabla = 'interactions'
nombre_columnas = ','.join(df_full.columns) # creo una string

# Bucle por filas y voy insertando en la tabla interactions de mi BBDD orcas
for i in range(df_full.shape[0]):
    valores = tuple(df_full.iloc[i].values)

    insert_query = f'insert into {nombre_tabla} ({nombre_columnas}) values {valores}'

    cursor.execute(insert_query)

In [24]:
insert_query

"insert into interactions (indice,date,Latitude,Longitude,Followed_GTOA_Protocol,Interaction_time,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour) values (152, Timestamp('2022-06-24 12:50:00'), 36.833333333333336, -8.916666666666666, 'No', '0-10', 'Sail', '10-12.5', 'No', 'Unknown', 'Unknown', 1, 1, 0, 'Spade', 'Sailing', '5 - 7', 'Calm', '3 - 4 (7 - 16 knots)', 'Day', '0 - 25%', 'Over 10', '200m+', 'On', 'On', 'White/light', 'Blue', 'No', 'No', 'The two orcas were nearby a buoy and as soon as they saw us, they came like a bullet. We stopped the boat and started the engine on reverse at full power. After 10 minutes hitting th

In [25]:
# Comprobamos

cursor.execute('select * from interactions limit 2;')

for x in cursor:
    print(x)

(0, datetime.datetime(2023, 11, 1, 22, 15), 32.7916, -9.90663, 'No', '0-10', 'Sail', '10-12.5', 'No', 'No', 'No', 1, 0, 0, 'Spade', 'Sailing', '5 - 7', 'Moderate', '5 - 6 (17 - 27 knots)', 'Night', '0 - 25%', 'Over 10', '200m+', 'On', 'On', 'White/light', 'Blue', 'No', 'No', "Orca interaction at 10:15pm on 01/11, 40 miles off the coast of Safi, on passage from Tangier to the Canary Islands. We were relaxing in the cockpit with a podcast playing quite loudly on a portable speaker. It was pitch black as the moon had not yet risen. A lone adult orca (to our knowledge) approached our yacht (Beneteau Oceanis 411) and surfaced about half a metre from our stern. We were only alerted to it's presence when it's blowhole spouted, roughly 0.5 metres off the left side of our stern, right next to our navigation light. It was so close to us that the water from its blowhole hit our chart plotter. We stood up to inspect and saw 1 very large (adult) Orca. We scrambled to grab our foghorn, and 2 x firec

In [26]:
# Comprobamos tamaño

cursor.execute('select count(*) from interactions;')

for x in cursor:
    print(x)

(153,)


Ya tenemos la BBDD creada en Workbench y hemos creado dentro de ella una tabla con la información de las interacciones.